In [1]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Action space: Discrete(2)
Observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [4]:
from itertools import count
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch

Модель Actor-Critic является дальнейшим развитием градиентов политики, в которой мы строим нейронную сеть для обучения как политике, так и оценкам вознаграждения. Сеть будет иметь два выхода (или вы можете рассматривать ее как две отдельные сети):

- Actor будет рекомендовать действия, предоставляя нам распределение вероятности состояния, как в градиентной модели политики.
- Critic оценивает, какова будет награда от этих действий. Он возвращает суммарное предполагаемое вознаграждение в будущем при данном состоянии.
Определим такую модель:

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Для этого нам потребуется немного изменить функции discounted_rewards и run_episode:

In [10]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        state = state[0]
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _,_ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()

Теперь запустим основной цикл обучения. Мы будем использовать ручной процесс обучения сети, вычисляя соответствующие функции потерь и обновляя параметры сети:

In [11]:
actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

Iteration: 0, Score: 21
Iteration: 1, Score: 19
Iteration: 2, Score: 60
Iteration: 3, Score: 9
Iteration: 4, Score: 11
Iteration: 5, Score: 22
Iteration: 6, Score: 22
Iteration: 7, Score: 18
Iteration: 8, Score: 10
Iteration: 9, Score: 11
Iteration: 10, Score: 18
Iteration: 11, Score: 15
Iteration: 12, Score: 13
Iteration: 13, Score: 12
Iteration: 14, Score: 22
Iteration: 15, Score: 17
Iteration: 16, Score: 14
Iteration: 17, Score: 8
Iteration: 18, Score: 21
Iteration: 19, Score: 12
Iteration: 20, Score: 11
Iteration: 21, Score: 21
Iteration: 22, Score: 13
Iteration: 23, Score: 33
Iteration: 24, Score: 13
Iteration: 25, Score: 35
Iteration: 26, Score: 12
Iteration: 27, Score: 16
Iteration: 28, Score: 14
Iteration: 29, Score: 30
Iteration: 30, Score: 10
Iteration: 31, Score: 20
Iteration: 32, Score: 30
Iteration: 33, Score: 46
Iteration: 34, Score: 32
Iteration: 35, Score: 29
Iteration: 36, Score: 30
Iteration: 37, Score: 21
Iteration: 38, Score: 16
Iteration: 39, Score: 30
Iteration: 4

In [12]:
env.close()